# London Crime Statistics Analysis

## Introduction

In this Jupyter Notebook we are going to look at an analysis of London Crime Statistics, in particular, stop and search data.

The idea of this report is to understand if the there are any trends in people being stopped and if certain objects of search have risen or declined in their frequency.

The data is sourced from Kaggle and can be found here: https://www.kaggle.com/sohier/london-police-records

## Data Cleaning and Data Wrangling

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

df = pd.read_csv('../input/london-stop-and-search.csv', low_memory=False)

df.head(5)

Here we are importting the packages we will use, along with the data. We are using a simple 'head' command to get an idea what it looks like. We can see here that there are some spaces in the titles of the columns and some uppercase characters, lets make them all lowercase and add in underscores to replace spaces. This will help us later on when putting together some graphs.

In [6]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

Okay perfect, lets look at the dataframes information to see what data types they are, and to confirm that the column names have been updated as we wished.

In [7]:
df.info()

Okay for this analysis there are a few columns we don't want and there are a load of columns with missing data. First of all lets remove the columns we dont want as I do not believe they will be relevant. You can see I am removing the 'outcome linked to search column here, this is becuase I want to see trends in who is being searched and not the resulting actions.

In [8]:
df.drop(['policing_operation', 'self-defined_ethnicity', 'part_of_a_policing_operation', 'latitude', 'longitude', 'outcome_linked_to_object_of_search', 'removal_of_more_than_just_outer_clothing'], axis=1, inplace=True)

So I have removed some columns, now lets clean the data up. But first let's see some statistics. Always helpful. Normally I would do some histograms here, but as this data is nearly all text based they wouldn't prove too useful.

In [9]:
df.describe()

Let's start the cleaning! First of all, lets change date to be in datetime instead of a string.

In [10]:
df['date'] = pd.to_datetime(df['date'])

In [11]:
df['date'] = df['date'].dt.date

And lets change the long winded name of 'Officer Defined Ethnicity' and let's use this as Ethnicity.

In [12]:
df.rename({'officer-defined_ethnicity':'ethnicity'}, axis=1, inplace=True)

In [13]:
df.head()

As we saw earlier, there were a lot of empty rows - I am just going to remove these as we cannot do anything such as 'fill with mean' as this is text data. If we also replaced it would skew results - first I will see how many we have.

In [14]:
df.isnull().sum()

In [15]:
df_clean = df.dropna() #remove all null values so we have clean dataset

In [16]:
df_clean.info() #quick check

We have created a new dataframe with the null values removed, called df_clean.

## Data Analysis

In [24]:
plot_month = df_clean.groupby('date').size().reset_index(name='number of outcomes').set_index('date')
plot_month.plot(kind='line', figsize=(20,10));

Oh wow - does not look like the 2015 data is all that useful here and barely registers. Let's remove this year.

In [19]:
#there is a lot of blank data for 2015, I am going to remove this from the dataframe
df['date'] = pd.to_datetime(df['date'])
df_clean = df[df['date'].dt.year != 2015]

In [21]:
df_clean = df_clean.dropna()

Time to plot again! This time let's make a better graph using matplotlib.

In [22]:
count_by_date = df_clean.groupby('date').size()

#z = np.polyfit(x, 1)
#p = np.poly1d(z)

plt.figure(figsize=(20,10))
plt.xlabel('Date')
plt.ylabel('Count of Stop and Search Record')
plt.title('Time series graph showing stop and search activity')
plt.plot(count_by_date);

Okay that looks better - I want to do some ethnicity analysis so let's see what values we have.

In [23]:
df_clean['ethnicity'].value_counts()

Let's plot a nice simple bar chart to plot the count.

In [386]:
df_clean['ethnicity'].value_counts().plot.bar(title='stop_and_search_by_ethnicity',figsize=(15,10));

Now let's plot by object of search - to see what is most common

In [387]:
df_clean['object_of_search'].value_counts().plot.bar(title='Stop and Search by Object of Search',figsize=(15,10));

So the most common thing is for a white person to get stopped under the objection of police finding controlled drugs.

Let's combine these two - and do a graph of both in grouped bar charts.

In [388]:
df_clean.groupby('ethnicity')['object_of_search'].value_counts().unstack(0).plot.bar(title='Object of search by Ethnicity', figsize=(15,10));

In [390]:
df_clean.groupby('object_of_search')['ethnicity'].value_counts().unstack(0).plot.bar(title='Ethnicity by Object of Search', figsize=(15,10));

This shows us that Controlled Drugs is the most common thing to get stopped for all ethnicities. With articles for criminal damage coming in second. We can also see that White people are ahead for the two main crimes here, but Black people are higher for firearm usage.

Let's remove all but the top two and do some analysis.

In [391]:
counts = df_clean['object_of_search'].value_counts()
counts
counts[counts > 5000]

Let's plot a pie chart of these two as a % of both of them aded together. To make this easier we will make some masks to filter just both the results we are looking for.

In [392]:
drugs = df_clean.object_of_search == 'Controlled drugs'
criminal_damage = df_clean.object_of_search == 'Articles for use in criminal damage'

In [393]:
drugs_p_crimdam = df_clean.object_of_search[criminal_damage].count() + df_clean.object_of_search[criminal_damage].count()

In [394]:
drugs_prc = df_clean.object_of_search[drugs].count() / drugs_p_crimdam
criminal_damage_prc = df_clean.object_of_search[criminal_damage].count() / drugs_p_crimdam

In [395]:
# this will plot a pie chart for us
labels = 'Controlled Drugs', 'Articles for use in criminal damage'
fracs = [drugs_prc, criminal_damage_prc]
explode = (0,0)
plt.axis("equal")
plt.title('Percentage Drug vs. Weapons search as pct of Drugs plus Weapons')
plt.pie(fracs, explode=explode, labels=labels, autopct='%.0f%%', shadow=True);

Lovely - we can see that drugs are the main cause of stopping by a ratio of almost 2/3rds to a 1/3rd

I want to see some Gender data now - lets order by the object of search. 

In [396]:
df_clean.groupby('object_of_search')['gender'].value_counts().unstack(0).plot.bar(title='Gender by Object of Search', figsize=(15,10));

Well males are the overwhelming majority here but the trend still continues for the controlled drug searches. To show how much a majority of males there is lets make a pie chart.

In [397]:
gender_male = df_clean.gender == 'Male'
gender_female = df_clean.gender == 'Female'
gender_other = df_clean.gender == 'Other'

In [398]:
gender_male_prc = df_clean.gender[gender_male].count() / len(df_clean)
gender_female_prc = df_clean.gender[gender_female].count() / len(df_clean)
gender_other_prc = df_clean.gender[gender_other].count() / len(df_clean)

In [399]:
# this will plot a pie chart for us
labels = 'Male', 'Female', 'Other'
fracs = [gender_male_prc, gender_female_prc, gender_other_prc]
explode = (0.0,0,0)
plt.axis("equal")
plt.title('Percentage of Stop and Search by Gender')
plt.pie(fracs, explode=explode, labels=labels, autopct='%.0f%%', shadow=True);

Yep so males really are a massive majority. Now I want to see data by age.

In [400]:
age_u10 = df_clean.age_range == 'under 10'
age_10 = df_clean.age_range == '10-17'
age_18 = df_clean.age_range == '18-24'
age_25 = df_clean.age_range == '25-34'
age_34 = df_clean.age_range == 'over 34'

In [401]:
age_u10_prc = df_clean.age_range[age_u10].count() / len(df_clean)
age_10_prc = df_clean.age_range[age_10].count() / len(df_clean)
age_18_prc = df_clean.age_range[age_18].count() / len(df_clean)
age_25_prc = df_clean.age_range[age_25].count() / len(df_clean)
age_34_prc = df_clean.age_range[age_34].count() / len(df_clean)

In [402]:
 # this will plot a pie chart for us
labels = 'Under 10', '10-17', '18-24', '25-34', '35+'
fracs = [age_u10_prc, age_10_prc, age_18_prc, age_25_prc, age_34_prc]
explode = (0,0,0,0,0)
plt.axis("equal")
plt.title('Percentage of Stop and Search by Age range')
plt.pie(fracs, explode=explode, labels=labels, autopct='%.0f%%', shadow=True);

This is more interesting - 20% of the dataset is from 10 to 17. This is very young! Overall the 10-17 and 18-24 groups make up 61%! This really shows who police are viewing as committing the crimes and not who IS committing crimes, but it is a interesting insight.

In [462]:
df_clean.groupby('object_of_search')['age_range'].value_counts().unstack(0).plot.bar(title='Age range by Object of Search', figsize=(15,10));

This is the first time we have seen a trend where a stop and search for controlled drugs is not at the top, and this is for 10-17 year olds. We can also see just how overwhelmingly 18-24 year olds are getting stopped for drug related crimes.

Finally lets look at some time series graph to see if any trends have increased or decreased over the time series. 

In [426]:
gender_count = df_clean.groupby('date')
gender_count = gender_count.gender.apply(pd.value_counts).unstack(-1).fillna(0)


gender_count.plot(kind='line',figsize=(20,10), title='Stop and Search Count by Gender');

Slight downward trend in males and females looks to stay steady. It looks like there are som seasonal shifts in teh data - espeically obvious between August 2016 to Jan 2017.

In [427]:
race_count = df_clean.groupby('date')
race_count = race_count.ethnicity.apply(pd.value_counts).unstack(-1).fillna(0)

race_count.plot(kind='line',figsize=(20,10), title='Stop and Search Count by Race');

I thought we would see some more variation here -  but the data all kinds of follows each other in terms of trends. There is not really a stand out trend here.

In [428]:
object_count = df_clean.groupby('date')
object_count = object_count.object_of_search.apply(pd.value_counts).unstack(-1).fillna(0)

object_count.plot(kind='line',figsize=(20,10), title='Stop and Search Count by Object of Search');

In [488]:
age_count = df_clean.groupby('date')
age_count = age_count.age_range.apply(pd.value_counts).unstack(-1).fillna(0)
age_count.plot(kind='line', figsize=(20,10), title='Stop and Search Count by Age');

Again the trends kind of follow each other here, apart from some sharp spikes in 'Anything used to threaten or harm'.

## Conclusion

In conclusion I think we can see that there are trends in the data. We can see the two main reasons for stopping someone is for either suspected possesion of a controlled substance, drugs; or to look for an article that could be used for criminal damage. These trends are the same throuhgout Race, Age and Gender. The only exception here is that for 10-17 year olds we are more likely too see a sarch carried out looking for articles for crimonal damage instead of controlled substances.

This was an interesting excercise and I would want to compare this data against other regions in the future, using information from https://data.police.uk/data/.